# Lab Manual for Project-1-PyPair

This lab manual provides an overview of the project, explains the functionality of each section, and guides you on how to run the game.

## Overview
Project-1-PyPair is a Python-based text adventure game where players fight demons, collect items, and progress through a storyline. The game demonstrates the use of Python classes, JSON handling, file operations, and modular programming.

## Key Components

### 1. **Player Classes**
- Located in `Player.py`.
- Defines the player character classes: `Mage`, `Warrior`, `Shadow`, and `Archer`.
- Each class has unique attributes and level-up mechanics.

#### Example Code:

In [ ]:
# filepath: ~~\project-1-pypair\Project_Main_Function\Player.py
class Mage(Player):
    def __init__(self, name):
        super().__init__(name, 80, 150, 60, 6, 10, 18)
        self.magic_power = 25  # Unique attribute
    def level_up(self):
        super().level_up()
        self.hp += 5
        self.mana += 15
        self.intelligence += 5

## How to Run the Game
1. Ensure all required files are in the project directory.
2. Open a terminal and navigate to the project folder.
3. Run the following command:
   ```bash
   python Project_Main_Function/Game_Start.py
   ```
4. Follow the on-screen instructions to play the game.

In [ ]:
# filepath: ~~\project-1-pypair\Project_Main_Function\Combat_System.py
def combat(player, demon, location):
    while player.hp > 0 and demon['hp'] > 0:
        action = input("Choose an action: (1) Attack, (2) Use Item, (3) Run: ").strip()
        if action == "1":
            damage = calculate_damage(player, demon)
            demon['hp'] -= damage
            print(f"{player.name} dealt {damage} damage to {demon['name']}!")
        elif action == "2":
            use_item(player)
        elif action == "3":
            print(f"{player.name} tries to run away!")
            return

## Logging
- Logging is configured in multiple files to track errors and game events.
- Logs are displayed in the terminal for debugging purposes.

In [ ]:
# filepath: ~~\project-1-pypair\Project_Main_Function\Combat_System.py
def use_item(player):
    print("Select an item to use:")
    for i, item in enumerate(player.inventory, start=1):
        print(f"{i}. {item['name']}")
    choice = int(input("Enter the item number to use: "))
    item = player.inventory[choice - 1]
    if 'effect' in item:
        apply_item_effect(player, item)
        player.inventory.pop(choice - 1)

### Storyline

In [ ]:
# filepath: ~~\project-1-pypair\Project_Main_Function\Combat_System.py
def progress_story(player, node, storyline):
    event = storyline[node]
    slow_print(event['text'])
    if 'event' in event and event['event'] == 'combat':
        demon = spawn_demon('tier_1')
        combat(player, demon, node)

## How to Run the Game
1. Ensure all required files are in the project directory.
2. Open a terminal and navigate to the project folder.
3. Run the following command:
   ```bash
   python Project_Main_Function/Game_Start.py
   ```
4. Follow the on-screen instructions to play the game.

## JSON Files
- **`Demons.json`**: Contains demon data categorized by tiers.
- **`Items.json`**: Contains item data categorized by rarity.
- **`Storyline.json`**: Defines the game's narrative structure.

These files are loaded dynamically during the game to provide flexibility and scalability.

## Logging
- Logging is configured in multiple files to track errors and game events.
- Logs are displayed in the terminal for debugging purposes.

## Contribution
- **Huynh Gia Phong Tat**: Core mechanics and storyline.
- **Namho Kye**: Item system and combat logic.